# MA9

1. Take a *exponential moving average* **EMA** of $9$ periods

   

* If the **EMA** has an *negative* inclination an becomes *positive*
  1. Wait for the **EMA** to cross the price *and*  to have a positive inclination
  2. Mark the candle where the crossing happened
  3. Send a **Buy** order for some next candle that is above the select candle's **high**
  4. Put a **Stop Loss** at the selected candle's **low**



* If the **EMA** has an *positive* inclination an becomes *negative*
  1. Wait for the **EMA** to cross the price *and*  to have a negative inclination
  2. Mark the candle where the crossing happened
  3. Send a **Sell** order for some next candle that is below the select candle's **low**
  4. Put a **Stop Loss** at the selected candle's **high**

In [32]:
import sys
import os

import chart_studio.plotly as plty
import plotly.graph_objs as gobjs

import plotly

sys.path.insert(0, os.path.abspath('../py'))
from Secrets import ReadSecrets


account = ReadSecrets()\
        .set_secrets_path('../secrets.json')\
        .set_default_accessors()\
        .access('plotly')['main_account']

plty.sign_in(account['username'], account['apiKey'])


from datetime import datetime as calendar
import numpy as np

from pandas_datareader import data as pdr
import pandas as pd
import yfinance as yf
yf.pdr_override()

from DataHelper import DataHelper

from plotly import tools

In [33]:
data, close, extractors = DataHelper.get_history_formatted(['AAPL', 'MSFT', '^GSPC', 'VALE3.SA'], calendar(2018, 1 ,3), calendar(2020, 2, 1))
msft = extractors['MSFT']()
vale3 = extractors['VALE3.SA']()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [34]:
def get_ema(window, prices):
    K = ( 2 / (window + 1))
    ma = prices.rolling(window=window).mean().dropna()

    data = pd.DataFrame(index=ma.index)
    data['Price'] = prices
    data['EMA'] = np.NaN

    data.EMA[0] = ma[1]

    for i in range(1, len(data)):
        data.EMA[i] = (data.Price[i] * K) + ((1 - K) * data.EMA[i-1]) 

    return data

In [35]:
window = 9

vale3['EMA9'] = get_ema(9, vale3.Close).EMA

vale3['EMA_method2'] = vale3.Close.ewm(alpha=1/window, adjust=False, min_periods=window).mean()


In [52]:
# get the "slope of EMA9 9"
dif = vale3.EMA - vale3.shift(1).EMA

# ema 9 crossed the candle (open/close price) and ema 9 has an ascendent slope  ?
vale3['mark_max'] = np.where((vale3.EMA < vale3.High) & (vale3.EMA > vale3.Low) & (dif > 0), vale3.High, 0)

# ema 9 crossed the candle (open/close price) and ema 9 has an descendent slope  ?
vale3['mark_min'] = np.where((vale3.EMA > vale3.Low) & (vale3.EMA < vale3.High) & (dif < 0), vale3.Low, 0)

# get start point
vale3['buy_start'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).mark_max, np.NaN)
vale3['sell_start'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).mark_min, np.NaN)

# set stop loss
vale3['buy_stop'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).Low, np.NaN)
vale3['sell_stop'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).High, np.NaN)

display(vale3.head(60))

,Open,High,Low,Close,Adj Close,Volume,EMA,mark_max,mark_min,buy_start,sell_start,buy_stop,sell_stop
Date,,,,,,,,,,,,,
2018-01-03,41.830002,41.880001,41.299999,41.470001,38.826576,12744200,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-04,41.810001,42.369999,41.520000,41.639999,38.985741,18433000,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-05,41.570000,42.290001,41.310001,42.290001,39.594303,15251300,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-08,42.400002,43.230000,42.400002,43.230000,40.474384,14542800,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-09,43.580002,43.750000,42.930000,43.070000,40.324589,15986200,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-10,42.740002,42.950001,42.419998,42.470001,39.762829,12149500,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-11,42.599998,43.299999,42.599998,43.299999,40.539925,10236300,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-12,42.950001,43.660000,42.750000,43.549999,40.773987,15014300,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN
2018-01-15,43.500000,43.720001,43.270000,43.470001,40.699089,7253400,42.813333,0.000000,0.000000,NaN,NaN,NaN,NaN


In [72]:
trace_ema = gobjs.Scatter(
                x=vale3.index,
                y=vale3.EMA,
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_candles = gobjs.Candlestick(x=vale3.index,
                    name="VALE3.SA",
                    open=vale3.Open,
                    high=vale3.High,
                    low=vale3.Low,
                    close=vale3.Close,
                    increasing_line_color= 'cyan', decreasing_line_color= 'gray')

trace_buy = gobjs.Scatter(
    x=vale3.index,
    y=vale3.buy_start,
    mode="markers",
    name="Buy",
    marker = dict(size = 8, color = 'green', symbol = 'triangle-up', 
                  line=dict(width=1.5, color='DarkSlateGrey')),
)

trace_sell = gobjs.Scatter(
    x=vale3.index,
    y=vale3.sell_start,
    mode="markers",
    name="Sell",
    marker = dict(size = 8, color = 'red', symbol = 'triangle-down', 
                  line=dict(width=1.5, color='DarkSlateGrey')),
)

trace_sell_stop = gobjs.Scatter(
    x=vale3.index,
    y=vale3.sell_stop,
    mode="markers",
    name="Sell Stop",
    marker = dict(size = 8, color = 'red', symbol = 'line-ew', 
                  line=dict(width=1.5, color='red')),
)

trace_buy_stop = gobjs.Scatter(
    x=vale3.index,
    y=vale3.buy_stop,
    mode="markers",
    name="Buy Stop",
    marker = dict(size = 8, color = 'green', symbol = 'line-ew', 
                  line=dict(width=1.5, color='green')),
)

trace_cross = gobjs.Scatter(
    x=vale3.index,
    y=np.where((vale3.mark_max + vale3.mark_min) > 0, vale3.EMA, np.NaN),
    mode="markers",
    name="Crossing",
    marker = dict(size = 3, color = 'gold', symbol = 'circle', 
                  line=dict(width=1.5, color='DarkSlateGrey')),
)



plty.iplot([trace_ema, trace_candles, trace_buy, trace_sell, trace_cross, trace_sell_stop, trace_buy_stop], filename='ma9')

# Other method for calculating the slope

In [73]:
vale3['EMA9'] = get_ema(9, vale3.Close).EMA
vale3['EMA1'] = get_ema(1, vale3.Close).EMA


buy_cross = (vale3.shift(1).EMA1 < vale3.shift(1).EMA9) & (vale3.EMA1 > vale3.shift(1).EMA9)
sell_cross = (vale3.shift(1).EMA1 > vale3.shift(1).EMA9) & (vale3.EMA1 < vale3.shift(1).EMA9)

# ema 9 crossed the candle (open/close price) and ema 9 has an ascendent slope  ?
vale3['mark_max'] = np.where((vale3.EMA9 < vale3.High) & (vale3.EMA9 > vale3.Low) & (buy_cross == True), vale3.High, 0)

# ema 9 crossed the candle (open/close price) and ema 9 has an descendent slope  ?
vale3['mark_min'] = np.where((vale3.EMA9 > vale3.Low) & (vale3.EMA9 < vale3.High) & (sell_cross == True), vale3.Low, 0)

# get start point
vale3['buy_start'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).mark_max, np.NaN)
vale3['sell_start'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).mark_min, np.NaN)

# set stop loss
vale3['buy_stop'] = np.where((vale3.Low < vale3.shift(1).mark_max) & (vale3.High > vale3.shift(1).mark_max), vale3.shift(1).Low, np.NaN)
vale3['sell_stop'] = np.where((vale3.Low < vale3.shift(1).mark_min) & (vale3.High > vale3.shift(1).mark_min), vale3.shift(1).High, np.NaN)

In [75]:
trace_ema = gobjs.Scatter(
                x=vale3.index,
                y=vale3.EMA9,
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_candles = gobjs.Candlestick(x=vale3.index,
                    name="VALE3.SA",
                    open=vale3.Open,
                    high=vale3.High,
                    low=vale3.Low,
                    close=vale3.Close,
                    increasing_line_color= 'cyan', decreasing_line_color= 'gray')

trace_buy = gobjs.Scatter(
    x=vale3.index,
    y=vale3.buy_start,
    mode="markers",
    name="Buy",
    marker = dict(size = 8, color = 'green', symbol = 'triangle-up', 
                  line=dict(width=1.5, color='DarkSlateGrey')),
)

trace_sell = gobjs.Scatter(
    x=vale3.index,
    y=vale3.sell_start,
    mode="markers",
    name="Sell",
    marker = dict(size = 8, color = 'red', symbol = 'triangle-down', 
                  line=dict(width=1.5, color='DarkSlateGrey')),
)

trace_sell_stop = gobjs.Scatter(
    x=vale3.index,
    y=vale3.sell_stop,
    mode="markers",
    name="Sell Stop",
    marker = dict(size = 8, color = 'red', symbol = 'line-ew', 
                  line=dict(width=1.5, color='red')),
)

trace_buy_stop = gobjs.Scatter(
    x=vale3.index,
    y=vale3.buy_stop,
    mode="markers",
    name="Buy Stop",
    marker = dict(size = 8, color = 'green', symbol = 'line-ew', 
                  line=dict(width=1.5, color='green')),
)

trace_cross = gobjs.Scatter(
    x=vale3.index,
    y=np.where((vale3.mark_max + vale3.mark_min) > 0, vale3.EMA, np.NaN),
    mode="markers",
    name="Crossing",
    marker = dict(size = 3, color = 'gold', symbol = 'circle', 
                  line=dict(width=1.5, color='DarkSlateGrey')),
)



plty.iplot([trace_ema, trace_candles, trace_buy, trace_sell, trace_cross, trace_sell_stop, trace_buy_stop], filename='ma9')

In [76]:
trace_short = gobjs.Scatter(
                x=vale3.index,
                y=vale3.EMA,
                name = "EMA(1)",
                line = dict(color = '#FF00CF'),
                opacity = 1)

trace_long = gobjs.Scatter(
                x=vale3.index,
                y=vale3.EMA9,
                name = "EMA(9)",
                line = dict(color = '#17BECF'),
                opacity = 1)

trace_candles = gobjs.Candlestick(x=vale3.index,
                    open=vale3.Open,
                    high=vale3.High,
                    low=vale3.Low,
                    close=vale3.Close)

data = [trace_short, trace_long, trace_candles]

fig = gobjs.Figure(data=data)

plty.iplot(fig, filename='cross-mm9-plot')